In [833]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import *
from collections import Counter
from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import torch

## 추천 시스템 모델링
1. 전처리
2. 토큰화, 벡터화(bert)
3. 사용자 입력 및 유사도 계산

In [866]:
#파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋.csv"
stopword_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/불용어리스트_한국어.txt"

In [867]:
df = pd.read_csv(df_file, encoding='CP949')
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head(5)

,꽃,월,계절,꽃말,설명,이미지
0,검은포플라,1,겨울,용기,당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고 해서 자기만족...,https://mblogthumb-phinf.pstatic.net/MjAyMjAxM...
1,군자란,1,겨울,고귀,君子蘭(군자란)은 이름 끝에 ‘란’이라고 되어있어서 난 종류일거라 생각하기도 하지만...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
2,나도풍란,1,겨울,인내,남부지역의 바위나 나무에 붙어사는 난이다. 보통 풍란 하면 소엽풍란을 말한다. 나도...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
3,노랑 히야신,1,겨울,승부,당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다. 엄격히 하는 것도 시간과 경...,https://img1.daumcdn.net/thumb/R720x0.q80/?sco...
4,노랑수선화,1,겨울,사랑에 답하여,당신은 추진력이 강한 운명을 타고 났습니다. 따라서 아무리 불가능해 보이는 사랑도 ...,https://blog.kakaocdn.net/dn/v1i65/btqRoRsgqCT...


In [868]:
#nlp처리 할 데이터만 가져오기
df_nlp = df[["꽃","꽃말", "설명"]]
print(df_nlp.shape)
df_nlp.head(5)

(845, 3)


,꽃,꽃말,설명
0,검은포플라,용기,당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고 해서 자기만족...
1,군자란,고귀,君子蘭(군자란)은 이름 끝에 ‘란’이라고 되어있어서 난 종류일거라 생각하기도 하지만...
2,나도풍란,인내,남부지역의 바위나 나무에 붙어사는 난이다. 보통 풍란 하면 소엽풍란을 말한다. 나도...
3,노랑 히야신,승부,당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다. 엄격히 하는 것도 시간과 경...
4,노랑수선화,사랑에 답하여,당신은 추진력이 강한 운명을 타고 났습니다. 따라서 아무리 불가능해 보이는 사랑도 ...


In [869]:
#꽃말과 설명 합치기
df_nlp['설명'] = df_nlp['꽃말'] + " " + df_nlp['설명']
df_nlp.drop("꽃말", axis=1, inplace=True)
df_nlp.head(5)

C:\Users\pc\AppData\Local\Temp\ipykernel_8840\3361795772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['꽃말'] + " " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_8840\3361795772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp.drop("꽃말", axis=1, inplace=True)


,꽃,설명
0,검은포플라,용기 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고 해서 자...
1,군자란,고귀 君子蘭(군자란)은 이름 끝에 ‘란’이라고 되어있어서 난 종류일거라 생각하기도 ...
2,나도풍란,인내 남부지역의 바위나 나무에 붙어사는 난이다. 보통 풍란 하면 소엽풍란을 말한다....
3,노랑 히야신,승부 당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다. 엄격히 하는 것도 시간...
4,노랑수선화,사랑에 답하여 당신은 추진력이 강한 운명을 타고 났습니다. 따라서 아무리 불가능해 ...


#### 1. 전처리

In [870]:
#특수문자 및 띄어쓰기 제거
import re

def remove_special_characters(text):
    # 특수문자 제거
    pattern = r'[^\w\s\.]' #문자,공백문자,마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' #중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)

df_nlp.head(10)

C:\Users\pc\AppData\Local\Temp\ipykernel_8840\2858685949.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)


,꽃,설명
0,검은포플라,용기 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다 그렇다고 해서 자기...
1,군자란,고귀 군자란은 이름 끝에 란이라고 되어있어서 난 종류일거라 생각하기도 하지만 난과는...
2,나도풍란,인내 남부지역의 바위나 나무에 붙어사는 난이다 보통 풍란 하면 소엽풍란을 말한다 나...
3,노랑 히야신,승부 당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다 엄격히 하는 것도 시간과...
4,노랑수선화,사랑에 답하여 당신은 추진력이 강한 운명을 타고 났습니다 따라서 아무리 불가능해 보...
5,노랑제비꽃,수줍은 사랑 노랑제비꽃을 탄생화로 태어난 당신은 용기 사랑 헌신을 갖고 있으니 겁쟁...
6,노루귀,인내 당신은 인내심이 강하고 아부를 싫어해 누구에게나 신뢰를 받지만 연애는 수동적인...
7,매쉬매리골드,반드시 오고야말 행복 주변 사람들이 행복해지면 자신도 역시 미소 지을 날이 옵니다 ...
8,미나리아재비,천진난만 부귀에 대한 욕망이 남들보다 월등히 강한당신은 자존심이 강해 다른 사람들 ...
9,미니 방울 수선화,자기애 자존심 고결 수선화는 튤립 히아신스와 같은 대표적인 구근식물로 다양한 품종이...


#### 2. 토큰화, 벡터화(bert)
[토큰화]
- bert의 토큰화 진행 시, BERT의 설계와 사전 학습된 방식에 맞추어 불용어 제거 없이 사용하는 것이 일반적으로 더 나은 성능을 발휘 </br>

[벡터화]
- bert를 통해 '꽃말', '설명' 칼럼 벡터 추출 -> 하나의 벡터로 결합
- 월, 계절 원핫인코딩 -> 하나의 벡터
- 총 2개의 벡터 생성 </br>

*문장 벡터의 차원이 훨씬 커서 원핫 인코딩 벡터의 상대적인 중요성이 낮아질 수 있기 때문

불용어 리스트: https://wikidocs.net/22530

In [873]:
# BERT 모델과 토크나이저 불러오기

model_name = 'monologg/kobert' # 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [877]:
# 문장을 임베딩 벡터로 변환하는 함수
def get_sentence_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True) #최대길이 초과 시 잘라내기, 작은 경우 패딩진행
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy() #.flatten() #2차원 -> 1차원 배열로 변환

In [878]:
#설명 벡터화
df_nlp['설명_벡터'] = df_nlp['설명'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) #데이터마다 (1, 768) 차원 벡터 생성

C:\Users\pc\AppData\Local\Temp\ipykernel_8840\724873025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명_벡터'] = df_nlp['설명'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) #데이터마다 (단어 개수, 768) 차원 벡터 생성 -> 1차원으로


In [ ]:
# type(df_nlp['꽃말_벡터'][1])
# pd.DataFrame(df_nlp['꽃말_벡터'][1])

In [927]:
#월과 계절 원핫인코딩
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['월', '계절']]).toarray()  # '월'과 '계절' 컬럼을 원핫인코딩
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['월', '계절'])) # 원핫인코딩된 결과를 새로운 데이터프레임으로 변환

# 원핫인코딩 결과를 '원핫인코딩' 컬럼으로 추가
df_nlp['원핫인코딩'] = encoded_df.apply(lambda row: list(row), axis=1)
df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array(x).reshape(1, 16)) #(16,1)을 (1,16)차원으로 변경

C:\Users\pc\AppData\Local\Temp\ipykernel_8840\2934671132.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['원핫인코딩'] = encoded_df.apply(lambda row: list(row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_8840\2934671132.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array(x).reshape(1, 16)) #(16,1)을 (1,16)차원으로 변경


In [932]:
# 결합 벡터 생성 (설명 벡터, 꽃말 벡터 결합)
def create_combined_vector(row):
    설명_벡터 = row['설명_벡터'] #(1,768)
    원핫인코딩 = row['원핫인코딩'] #(1,16)

    combined_array = np.concatenate((설명_벡터, 원핫인코딩), axis=1) #(1,784) 

    return combined_array

df_nlp['최종_벡터'] = df_nlp.apply(create_combined_vector, axis=1)

C:\Users\pc\AppData\Local\Temp\ipykernel_8840\2624279429.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['최종_벡터'] = df_nlp.apply(create_combined_vector, axis=1)


In [871]:
# with open(stopword_file, 'r', encoding='utf-8') as file:
#     stopwords = file.read() # 파일 내용 읽기
# print(stopwords)

# #트큰화 (불용어 제외) - 빈도수 파악 위해
# okt = Okt()

# word_list=[]
# for i in df_nlp['설명']:
#     word_tokens = okt.morphs(i) #형태소 분석

#     for j in word_tokens:
#         if j not in stopwords: 
#             word_list.append(j)
            
# word_list

# #빈도수 확인
# frequent = Counter(word_list).most_common()
# frequent[:20]

# #불용어 재구축
# stopwords_re = "꽃 입니다 줄기 있는 이나 품종 처럼 식물 많이 에서는"
# stopwords_re  = stopwords_re +' ' + stopwords
# stopwords_re = stopwords_re.replace('의지하여', '')
# stopwords_re

In [872]:
# #불용어 제거 함수화
# def remove_stopword(text):
#     word_tokens = okt.morphs(text) #토큰화
    
#     a= []
#     for i in word_tokens:
#         if i not in stopwords_re: #불용어가 아닌 단어만 넣기
#             a.append(i)

#     return a

# #불용어가 제거된 칼럼 생성
# df_nlp['설명_불용어제거'] = df_nlp['설명'].apply(remove_stopword)
# df_nlp.head(5)

# #형용사, 명사 등만 남기고 불용어 제거
# def extract_nouns_adjectives(text):
#     pos_tagged = okt.pos(text) #형태소 추출

#     nouns = [word for word, pos in pos_tagged if pos == 'Noun']
#     adjectives = [word for word, pos in pos_tagged if pos == 'Adjective']
#     a = nouns + adjectives
   
#     a_re=[]
#     for i in a:
#         if i not in stopwords_re: 
#             a_re.append(i)
#     return a_re

# #불용어 제거 및 형용사, 명사 형태소만 남긴 칼럼 생성
# df_nlp['설명_명사,형용사'] = df_nlp['설명'].apply(extract_nouns_adjectives)
# df_nlp[['설명', '설명_불용어제거', '설명_명사,형용사']].head(5)

# #트큰화 (불용어 제외, 명사와 형용사만 추출) - 빈도수 파악 위해
# word_list= []
# for i in df_nlp['설명_명사,형용사']:
#     for j in i:
#         if j not in stopwords_re: #불용어가 아닌 것만 넣기
#             word_list.append(j)

# #빈도수 확인
# frequent = Counter(word_list).most_common()
# frequent[:20]

# #불용어 재구축
# stopwords_re2 = "좋은 이름 연출 대표 사용 있습니다"
# stopwords_re  = stopwords_re2 +' ' + stopwords_re
# stopwords_re

# #불용어 제거 및 형용사, 명사 형태소만 남긴 칼럼 재생성 (위 불용어 적용하여)
# df_nlp['설명_명사,형용사'] = df_nlp['설명'].apply(extract_nouns_adjectives)
# df_nlp[['설명', '설명_불용어제거', '설명_명사,형용사']].head(5)

# #리스트 내에 중복되는 부분 제거
# from collections import OrderedDict

# df_nlp['설명_명사,형용사'] = df_nlp['설명_명사,형용사'].apply(lambda x: list(OrderedDict.fromkeys(x))) #순서 섞이지 않도록 고정
# df_nlp[['설명', '설명_불용어제거', '설명_명사,형용사']].head(5)

#### 4. 사용자 입력 및 유사도 계산 
- 사용자 입력 벡터화
- 유사도 계산

 > 사용자 입력에서 월이나 계절 내용이 없으면 사용자가 입력한 날짜를 추출해서 가져오기

In [934]:
#이미지 칼럼 생성
df_nlp['이미지'] = df['이미지']

In [935]:
df_nlp.head(5)

,꽃,설명,설명_벡터,원핫인코딩,최종_벡터,이미지
0,검은포플라,용기 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다 그렇다고 해서 자기...,"[[-0.08729376643896103, 0.050417497754096985, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.08729376643896103, 0.050417497754096985, ...",https://mblogthumb-phinf.pstatic.net/MjAyMjAxM...
1,군자란,고귀 군자란은 이름 끝에 란이라고 되어있어서 난 종류일거라 생각하기도 하지만 난과는...,"[[0.12972727417945862, -0.13232304155826569, -...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.12972727417945862, -0.13232304155826569, -...",https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
2,나도풍란,인내 남부지역의 바위나 나무에 붙어사는 난이다 보통 풍란 하면 소엽풍란을 말한다 나...,"[[0.0757506787776947, -0.0020380616188049316, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0757506787776947, -0.0020380616188049316, ...",https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
3,노랑 히야신,승부 당신은 언제나 엄격하지 않으면 못견뎌 하는 성격입니다 엄격히 하는 것도 시간과...,"[[0.186703622341156, -0.13133515417575836, -0....","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.186703622341156, -0.13133515417575836, -0....",https://img1.daumcdn.net/thumb/R720x0.q80/?sco...
4,노랑수선화,사랑에 답하여 당신은 추진력이 강한 운명을 타고 났습니다 따라서 아무리 불가능해 보...,"[[0.12058863043785095, 0.015314951539039612, -...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.12058863043785095, 0.015314951539039612, -...",https://blog.kakaocdn.net/dn/v1i65/btqRoRsgqCT...


In [936]:
# 사용자 입력 텍스트 분석 함수(월, 계절)
def extract_month_season(text):
    months = {
        '1월': 1, '2월': 2, '3월': 3, '4월': 4, '5월': 5, '6월': 6,
        '7월': 7, '8월': 8, '9월': 9, '10월': 10, '11월': 11, '12월': 12
    }
    seasons = {'봄': '봄', '여름': '여름', '가을': '가을', '겨울': '겨울'}
    
    month = None
    season = None
    
    for key, value in months.items():
        if key in text:
            month = value
            break
    
    for key in seasons.keys():
        if key in text:
            season = key
            break
    
    return month, season

In [955]:
#사용자 입력을 최종 벡터화 
def get_user_input_vector(user_input):
    input = remove_special_characters(user_input) #특수문자 제거
    
    # 토큰화 및 벡터화
    input_embeddings = get_sentence_embedding(input, tokenizer, model) #(1,768)
    month, season = extract_month_season(user_input)

    # 원핫 벡터 생성
    user_onehot_vector = np.zeros(len(encoder.get_feature_names_out(['월', '계절'])))
    if month is not None:
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is not None:
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
    user_onehot_vector = user_onehot_vector.reshape(1, 16) #(1,16)

    #벡터 결합
    user_vector = np.concatenate((input_embeddings, user_onehot_vector), axis=1) #(1,784)
    return user_vector

In [1016]:
# 추천 시스템 함수 (코사인 유사도 기반)
from sklearn.metrics.pairwise import cosine_similarity

def recommend_flower(user_input, tokenizer, model):
    user_vector = get_user_input_vector(user_input) #사용자 입력을 최종 벡터화

    #코사인 유사도 산출
    df_nlp['유사도'] = df_nlp['최종_벡터'].apply(lambda x: cosine_similarity(user_vector, x)[0][0])
    df_nlp['유사도'] = df_nlp['유사도'].astype(float) #숫자형으로 변환

    # 유사도를 기준으로 상위 3개의 꽃을 선택하고 중복된 꽃을 제거
    top3 = df_nlp.nlargest(3, '유사도').drop_duplicates(subset='꽃')

    # 만약 중복 제거 후 3개의 꽃이 되지 않는 경우, 다시 nlargest로 채우기
    if top3.shape[0] < 3:
        additional_top = df_nlp.nlargest(20, '유사도')  # 상위 10개 정도를 선택
        additional_top = additional_top[~additional_top['꽃'].isin(top3['꽃'])]
        top3 = pd.concat([top3, additional_top]).nlargest(3, '유사도').drop_duplicates(subset='꽃')

    return top3[['꽃', '유사도', '이미지']]

In [1011]:
#성능 테스트 해보기

#"여자친구 생일이 6월 10일이야"
#"친구가 어제 드디어 취업을 해서 꽃을 선물하려해"
#"어제 여자친구랑 싸워서 화해를 하려해"
#"부모님 생신이여서 꽃을 선물하고 싶은데 화려한 꽃으로 축하하고 싶어"

user_input = "여자친구 생일이 6월 10일이야"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,유사도,이미지
498,카네이션,0.896578,https://image.kmib.co.kr/online_image/2024/050...
496,초롱꽃,0.894089,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
457,에그타르트 장미,0.893676,https://file.honestflower.kr/media/images/ingr...


초롱꽃, 에그타르트 장미는 6월 꽃임 (카네이션은 어떤 부분에서 나온지 모르겠음)

In [1012]:
user_input = "친구가 어제 드디어 취업을 해서 꽃을 선물하려해"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,유사도,이미지
180,라일락,0.959519,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...
639,로사캠피온,0.959185,https://mblogthumb-phinf.pstatic.net/MjAxNzA4M...
651,스타티스,0.958821,https://s3.ap-northeast-2.amazonaws.com/om-pub...


취업, 선물, 친구라는 단어가 존재하지 않아서 아마 유사한 의미로 대체된 듯

In [1013]:
user_input = "어제 여자친구랑 싸워서 화해를 하려해"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,유사도,이미지
794,서양모과,0.949536,https://img1.daumcdn.net/thumb/R800x0/?scode=m...
89,칼미아,0.948300,https://encrypted-tbn0.gstatic.com/images?q=tb...
664,진달래,0.947336,https://pds.joongang.co.kr/news/component/html...


칼미아는 용기라는 의미가 있음

In [1014]:
user_input = "부모님 생신이여서 꽃을 선물하고 싶은데 화려한 꽃으로 축하하고 싶어"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,유사도,이미지
635,능소화,0.960701,https://encrypted-tbn1.gstatic.com/images?q=tb...
664,진달래,0.959587,https://pds.joongang.co.kr/news/component/html...
89,칼미아,0.959559,https://encrypted-tbn0.gstatic.com/images?q=tb...


능소화(명예), 진달래(사랑의 기쁨), 칼미아(용기) -> 전혀 안맞는데?

In [1018]:
user_input = "어버이날"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,유사도,이미지
802,초롱꽃,0.883691,https://t2.daumcdn.net/thumb/R720x0.fjpg/?fnam...
131,수선화,0.879561,https://i.namu.wiki/i/fk6EUlXQiNYbuUekI4XjT9v9...
248,덴파레,0.878278,https://file.honestflower.kr/media/images/ingr...


In [1030]:
# 예시 사용자 입력
user_input = "나는 1월에 겨울에 어울리는 용기 있는 꽃을 찾고 있어"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,유사도,이미지
834,온시디움,0.950974,https://file.honestflower.kr/media/images/ingr...
587,세일러문 해바라기,0.945705,https://file.honestflower.kr/media/images/ingr...
843,호접란,0.944824,https://file.honestflower.kr/media/images/ingr...


온시디움, 호접란은 모두 겨울 꽃